<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/IP-ScrappingClean/blob/master/ip%E4%BB%A3%E7%90%86%E6%8A%93%E5%8F%96%E6%B8%85%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time 
import numpy as np
import pandas as pd
from google.colab import files
import datetime
import json
import random

In [ ]:
"""html 用来连接网站以及返回html"""
class ConnectHtml:

  def __init__(self,url):

    self.url = url
    self.html = self.connect()

  def connect(self):

    kv = {'user-agent':
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    try:
      response = requests.get(self.url, timeout = 10, headers = kv)
      response.raise_for_status()
      response.encoding = response.apparent_encoding
    except Exception as e:
      print(f"Exception occur: {e}")
    else:
      return response.text

  def parser(self):

    soup = bs(self.html,'html.parser')
    return soup

In [ ]:
class IpGetter:

  def __init__(self):
    self.url = 'https://www.kuaidaili.com/free/inha/'
    self.check_url = 'https://www.ip.cn/'
    self.ip_list = []

  def get_available_ip(self, ip_address, ip_port):
    """检测IP地址是否可用"""
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
    ip_url_next = '://' + ip_address + ':' + ip_port
    proxies = {'http': 'http' + ip_url_next}
    try:
      r = requests.get(self.check_url, headers=header, proxies=proxies, timeout=3)
      html = r.text
    except:
      print('fail-%s' % ip_address)
    else:
      print('success-%s' % ip_address)
      soup = bs(html, 'lxml')
      div = soup.find(class_='well')
      if div:
          print(div.text)
      ip_info = [ip_address,ip_port]
      self.ip_list.append(ip_info)

  def scrapping(self):

    for i in range(1,2):
      newurl = self.url+f'{str(i)}/'
      myconnect = ConnectHtml(newurl)
      soup = myconnect.parser()

      tbody = soup.find('tbody')
      for tr in tbody('tr'):
        tds = tr.find_all('td')
        self.get_available_ip(tds[0].string,tds[1].string)
      
      print(f'Scrapping for {str(i)}th page')
      t = np.random.randint(3,8)
      time.sleep(t)
    


In [ ]:
test_getter = IpGetter()
test_getter.scrapping()

success-163.204.245.172
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-125.108.93.141
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-113.121.67.66
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-113.195.235.159
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-114.239.172.17
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-118.113.247.182
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-113.195.235.101
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-47.94.89.87
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-183.166.111.95
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-175.42.158.68
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States
success-124.200.36.118
您现在的 IP：35.237.66.235所在地理位置：美国 GoogleGeoIP: Virginia, United States


In [ ]:
my_IP = test_getter.ip_list

In [ ]:
frame_data = pd.DataFrame(my_IP)

In [ ]:
frame_data.to_csv('Ip_address.csv')

In [ ]:
files.download('Ip_address.csv')

In [ ]:
ip_random = -1

def get_html(url):
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'
    }
    global ip_random
    ip_rand, proxies = get_proxie(ip_random)
    print(proxies)
    try:
        request = requests.get(url=url, headers=header, proxies=proxies, timeout=3)
    except:
        request_status = 500
    else:
        request_status = request.status_code
    print(request_status)
    while request_status != 200:
        ip_random = -1
        ip_rand, proxies = get_proxie(ip_random)
        print(proxies)
        try:
            request = requests.get(url=url, headers=header, proxies=proxies, timeout=3)
        except:
            request_status = 500
        else:
            request_status = request.status_code
        print(request_status)
    ip_random = ip_rand
    request.encoding = 'gbk'
    html = request.content
    print(html)
    return html

def get_proxie(random_number):
    with open('Ip_address.csv', 'r') as file:
        ip_list = json.load(file)
        if random_number == -1:
            random_number = random.randint(0, len(ip_list) - 1)
        ip_info = ip_list[random_number]
        ip_url_next = '://' + ip_info['address'] + ':' + ip_info['port']
        proxies = {'http': 'http' + ip_url_next, 'https': 'https' + ip_url_next}
        return random_number, proxies


In [ ]:
#示例
ip_url_next = '://' + '124.200.36.118' + ':' + '40188'
proxies = {'http': 'http' + ip_url_next}
url = 'https://www.ssense.com/en-us/men'
headers = {
'Accept': '*/*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Connection': 'keep-alive',
'Host' : 'bam.nr-data.net',
'Cookie': 'JSESSIONID=6d416ef337bd8764',
'Referer': 'https://www.ssense.com/en-us/men',
'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
try:
    request = requests.get(url=url, headers=header, proxies=proxies, timeout=3)
except:
    request_status = 500
else:
    request_status = request.status_code
print(request_status)